# Rozliczanie podatków od kryptowalut w Polsce

## Konfiguracja

Konfigurację zacznij od wprowadzenia swoich kluczy API do giełd w pliku .env.
Plik konfiguracyjny stworzysz na bazie pliku .env.example.

In [15]:
ROK = 2024
FIAT_CURRENCY_SYMBOLS = {'PLN', 'USD', 'EUR', 'CHF', 'GBP'}

## Importy

In [16]:
from kryptorozliczator.exchange_interfaces.zonda_interface import ZondaInterface
from kryptorozliczator.exchange_interfaces.exchange_interface import ExchangeInterface

import pandas as pd
import requests
from datetime import datetime, timedelta

interfaces = [ExchangeInterface('zonda'), ExchangeInterface('bitfinex')]

## Przydatne funkcje

In [17]:
def filter_pln_pair(symbol):
    for symbol_part in symbol.split('/'):
        if symbol_part == 'PLN':
            return True
    return False

def filter_fiat_pair(symbol):
    for symbol_part in symbol.split('/'):
        if symbol_part in FIAT_CURRENCY_SYMBOLS:
            return True
    return False

def filter_currency_pair(symbol, currency_symbol):
    for symbol_part in symbol.split('/'):
        if symbol_part == currency_symbol:
            return True
    return False

def get_nbp_exchange_rate(currency_code, date):
    """
    Get exchange rate from NBP API for a given currency and date.
    Returns exchange rate or 1.0 for PLN.
    """
    if currency_code == 'PLN':
        return 1.0
        
    # NBP API requires uppercase currency codes
    currency_code = currency_code.upper()
    
    # Format date as YYYY-MM-DD
    date_str = date.strftime('%Y-%m-%d')
    
    # NBP API endpoint
    url = f"http://api.nbp.pl/api/exchangerates/rates/A/{currency_code}/{date_str}/?format=json"
    # print(url)
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data['rates'][0]['mid']
        elif response.status_code == 404:
            # If rate not found for given date, try previous day
            yesterday = date - timedelta(days=1)
            return get_nbp_exchange_rate(currency_code, yesterday)
    except Exception as e:
        raise Exception(f"Failed to get exchange rate for {currency_code}: {str(e)}")


## Pobranie historii transakcji

In [18]:
all_transactions_df = pd.DataFrame()

for interface in interfaces:
    print(f"Pobieranie transakcji z {interface.exchange_name}")
    transactions = interface.get_transaction_history(ROK)
    transactions_df = pd.DataFrame(transactions)
    transactions_df["exchange"] = interface.exchange_name
    all_transactions_df = pd.concat([all_transactions_df, transactions_df])


Pobieranie transakcji z zonda
Fetching trades since 2024-01-01T00:00:00.000Z...
Reached end of relevant period or fetched less than limit.
Fetched 10 total trades for the year 2024.
Pobieranie transakcji z bitfinex
Fetching trades since 2024-01-01T00:00:00.000Z...
Reached end of relevant period or fetched less than limit.
Fetched 18 total trades for the year 2024.


## Wyświetlenie trazakcji

In [19]:
## Display transactions as a table
display(all_transactions_df)

## Display transaction in print
# print(all_transactions_df)


,id,order,timestamp,datetime,symbol,type,side,price,amount,cost,takerOrMaker,fee,info,fees,exchange
0,bb916ef1-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,20.1800,6.441520,129.989876,taker,"{'currency': 'RENDER', 'cost': 0.01288305}","{'id': 'bb916ef1-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.01288305}]",zonda
1,bb916ef2-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,20.1900,43.960000,887.552400,taker,"{'currency': 'RENDER', 'cost': 0.08792}","{'id': 'bb916ef2-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.08792}]",zonda
2,bb916ef3-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,20.2000,119.000000,2403.800000,taker,"{'currency': 'RENDER', 'cost': 0.238}","{'id': 'bb916ef3-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.238}]",zonda
3,bb919604-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,21.1000,1.523223,32.140000,taker,"{'currency': 'RENDER', 'cost': 0.00304645}","{'id': 'bb919604-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.00304645}]",zonda
4,bb919605-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,21.1100,2.000000,42.220000,taker,"{'currency': 'RENDER', 'cost': 0.004}","{'id': 'bb919605-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.004}]",zonda
5,bb919606-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,21.1200,2.000000,42.240000,taker,"{'currency': 'RENDER', 'cost': 0.004}","{'id': 'bb919606-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.004}]",zonda
6,bb919607-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,21.1400,69.161306,1462.070000,taker,"{'currency': 'RENDER', 'cost': 0.13832262}","{'id': 'bb919607-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.13832262}]",zonda
7,732a3275-9fa5-11ef-a0fa-0242ac110007,732a3274-9fa5-11ef-a0fa-0242ac110007,1731271842234,2024-11-10T20:50:42.234Z,ETH/PLN,limit,buy,12630.0000,0.395883,4999.999890,taker,"{'currency': 'ETH', 'cost': 0.00178148}","{'id': '732a3275-9fa5-11ef-a0fa-0242ac110007',...","[{'currency': 'ETH', 'cost': 0.00178148}]",zonda
8,7678b8e9-a1cf-11ef-a0fa-0242ac110007,7678b8e8-a1cf-11ef-a0fa-0242ac110007,1731509788970,2024-11-13T14:56:28.970Z,ETH/PLN,limit,buy,13658.5000,0.732147,10000.029936,taker,"{'currency': 'ETH', 'cost': 0.00329467}","{'id': '7678b8e9-a1cf-11ef-a0fa-0242ac110007',...","[{'currency': 'ETH', 'cost': 0.00329467}]",zonda
9,89fe087a-b3f9-11ef-86d5-0242ac110005,fba763f7-b3f7-11ef-86d5-0242ac110005,1733506981512,2024-12-06T17:43:01.512Z,BTC/PLN,limit,sell,401900.0000,0.100000,40190.000000,maker,"{'currency': 'PLN', 'cost': 120.57}","{'id': '89fe087a-b3f9-11ef-86d5-0242ac110005',...","[{'currency': 'PLN', 'cost': 120.57}]",zonda


## Zapis transakcji do CSV

In [20]:
# Creata a directory for the CSV files in home directory ~/kryptorozliczator/csv
import os

csv_dir = os.path.expanduser("~/kryptorozliczator/csv")
os.makedirs(csv_dir, exist_ok=True)

# Create a CSV file for the transactions
csv_file = os.path.join(csv_dir, "transactions.csv")
all_transactions_df.to_csv(csv_file, index=False)

# Display the CSV file
# display(pd.read_csv(csv_file))

## Obliczenie przychodów i kosztów

In [21]:
def calculate_transaction_value_and_fee(row: pd.Series):
    currency_symbol, base_currency_symbol = row['symbol'].split('/')
    transaction_date = datetime.fromtimestamp(row['timestamp']/1000)
    transaction_value_fiat = row['cost']
    if base_currency_symbol == 'PLN':
        transaction_value_pln = transaction_value_fiat
    elif base_currency_symbol in FIAT_CURRENCY_SYMBOLS:
        exchange_rate = get_nbp_exchange_rate(base_currency_symbol, transaction_date)
        transaction_value_pln = transaction_value_fiat * exchange_rate
    else:
        raise ValueError(f"Unsupported currency symbol: {row['symbol']}")

    fee = row['fee']
    fee_currency = fee['currency']
    fee_cost = float(fee['cost'])
    print(f"Fee currency: {fee_currency}")
    if fee_currency == 'PLN':
        fee_value_pln = fee_cost
        fee_value_fiat = fee_cost
    elif fee_currency in FIAT_CURRENCY_SYMBOLS:
        exchange_rate = get_nbp_exchange_rate(fee_currency, transaction_date)
        fee_value_pln = fee_cost * exchange_rate
        fee_value_fiat = fee_cost
    else: # fee in crypto, convert to fiat, then to PLN
        crypto_to_fiat_exchange_rate = row['price']
        fee_value_fiat = fee_cost * crypto_to_fiat_exchange_rate
        if base_currency_symbol == 'PLN':
            fee_value_pln = fee_value_fiat
        elif base_currency_symbol in FIAT_CURRENCY_SYMBOLS:
            exchange_rate = get_nbp_exchange_rate(base_currency_symbol, transaction_date)
            fee_value_pln = fee_value_fiat * exchange_rate
        else:
            raise ValueError(f"Unsupported currency symbol: {row['symbol']}")
    return {'transaction_value_fiat': transaction_value_fiat,
            'transaction_value_pln': transaction_value_pln,
            'fee_value_fiat': fee_value_fiat,
            'fee_value_pln': fee_value_pln}

def calculate_transaction_totals_for_single_currency(df, currency_symbol):
    # Choose only transactions in the pair to fiat currency, the rest don't affect tax
    df = df[df['symbol'].apply(lambda x: filter_currency_pair(x, currency_symbol))]

    # Grupowanie transakcji na kupno i sprzedaż
    buys = df[df['side'] == 'buy']
    sells = df[df['side'] == 'sell']

    total_buy_cost_pln = 0
    total_buy_cost_original_currency = 0
    total_sell_revenue_pln = 0
    total_sell_revenue_original_currency = 0
    total_fee_fiat = 0
    total_fee_pln = 0
    for _, row in buys.iterrows():
        transaction_value_and_fee = calculate_transaction_value_and_fee(row)
        total_buy_cost_pln += transaction_value_and_fee['transaction_value_pln']
        total_buy_cost_original_currency += transaction_value_and_fee['transaction_value_fiat']
        total_fee_fiat += transaction_value_and_fee['fee_value_fiat']
        total_fee_pln += transaction_value_and_fee['fee_value_pln']

    for _, row in sells.iterrows():
        transaction_value_and_fee = calculate_transaction_value_and_fee(row)
        total_sell_revenue_pln += transaction_value_and_fee['transaction_value_pln']
        total_sell_revenue_original_currency += transaction_value_and_fee['transaction_value_fiat']
        total_fee_fiat += transaction_value_and_fee['fee_value_fiat']
        total_fee_pln += transaction_value_and_fee['fee_value_pln']
    
    return {'total_buy_cost_pln': total_buy_cost_pln,
            'total_buy_cost_original_currency': total_buy_cost_original_currency,
            'total_sell_revenue_pln': total_sell_revenue_pln,
            'total_sell_revenue_original_currency': total_sell_revenue_original_currency,
            'total_fee_fiat': total_fee_fiat,
            'total_fee_pln': total_fee_pln}

total_buy_cost_pln = 0
total_sell_revenue_pln = 0
total_fee_pln = 0
for currency_symbol in FIAT_CURRENCY_SYMBOLS:
    totals = calculate_transaction_totals_for_single_currency(all_transactions_df, currency_symbol)
    currency_summary_df = pd.DataFrame({
        'Kategoria': [f'Koszt zakupu (PLN)', f'Przychód ze sprzedaży (PLN)', f'Opłaty (PLN)', f'Koszt w {currency_symbol}', f'Przychód w {currency_symbol}'],
        'Wartość (PLN)': [
            totals['total_buy_cost_pln'],
            totals['total_sell_revenue_pln'],
            totals['total_fee_pln'],
            totals['total_buy_cost_original_currency'],
            totals['total_sell_revenue_original_currency']
        ]
    })
    print(f"\nPodsumowanie dla waluty {currency_symbol}:")
    display(currency_summary_df)

    total_buy_cost_pln += totals['total_buy_cost_pln']
    total_sell_revenue_pln += totals['total_sell_revenue_pln']
    total_fee_pln += totals['total_fee_pln']

summary_df = pd.DataFrame({
    'Category': ['Koszt zakupu', 'Przychód ze sprzedaży', 'Opłaty', 'Koszt całkowity', 'Zysk całkowity'],
    'Value (PLN)': [
        total_buy_cost_pln,
        total_sell_revenue_pln, 
        total_fee_pln,
        total_buy_cost_pln + total_fee_pln,
        total_sell_revenue_pln - total_buy_cost_pln - total_fee_pln
    ]
})
print("\nSummary:")
display(summary_df)


Podsumowanie dla waluty EUR:


,Kategoria,Wartość (PLN)
0,Koszt zakupu (PLN),0
1,Przychód ze sprzedaży (PLN),0
2,Opłaty (PLN),0
3,Koszt w EUR,0
4,Przychód w EUR,0


Fee currency: BTC
Fee currency: BTC
Fee currency: BTC
Fee currency: BTC
Fee currency: BTC
Fee currency: BTC

Podsumowanie dla waluty USD:


,Kategoria,Wartość (PLN)
0,Koszt zakupu (PLN),5802.708471
1,Przychód ze sprzedaży (PLN),59364.522424
2,Opłaty (PLN),43.228826
3,Koszt w USD,1435.470000
4,Przychód w USD,14688.588000


Fee currency: RENDER
Fee currency: RENDER
Fee currency: RENDER
Fee currency: RENDER
Fee currency: RENDER
Fee currency: RENDER
Fee currency: RENDER
Fee currency: ETH
Fee currency: ETH
Fee currency: PLN

Podsumowanie dla waluty PLN:


...

,Kategoria,Wartość (PLN)
0,Koszt zakupu (PLN),0
1,Przychód ze sprzedaży (PLN),0
2,Opłaty (PLN),0
3,Koszt w GBP,0
4,Przychód w GBP,0



Podsumowanie dla waluty CHF:


,Kategoria,Wartość (PLN)
0,Koszt zakupu (PLN),0
1,Przychód ze sprzedaży (PLN),0
2,Opłaty (PLN),0
3,Koszt w CHF,0
4,Przychód w CHF,0



Summary:


,Category,Value (PLN)
0,Koszt zakupu,25802.750574
1,Przychód ze sprzedaży,99554.522424
2,Opłaty,241.299193
3,Koszt całkowity,26044.049767
4,Zysk całkowity,73510.472657
